In [2]:
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import pickle
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import xgboost as xgb
print("XGBoost version:", xgb.__version__)
from itertools import combinations
import itertools

XGBoost version: 1.6.1


In [3]:
# ====================================================
# Directory settings
# ====================================================
import os
import pathlib

expt_name = "xgb_BOBW_5_fold"
OUTPUT_DIR = f'../{expt_name}/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
pathlib.path = OUTPUT_DIR


In [4]:

# ====================================================
# Configurations
# ====================================================
class CFG:
    input_dir = '../input/'
    seed = 42
    n_folds = 5
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'

# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)


In [83]:
# ====================================================
# Read data
# ====================================================
train = pd.read_parquet(CFG.input_dir + 'train_fe_v3.parquet')
test = pd.read_parquet(CFG.input_dir + 'test_fe_v3.parquet')

In [84]:
train.replace([np.inf, -np.inf], np.nan, inplace=True)
test.replace([np.inf, -np.inf], np.nan, inplace=True)

In [85]:
train.head()

,customer_ID,P_2_first,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_first,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,B_1_first,B_1_mean,B_1_std,B_1_min,B_1_max,B_1_last,B_2_first,B_2_mean,B_2_std,B_2_min,B_2_max,B_2_last,R_1_first,R_1_mean,R_1_std,R_1_min,R_1_max,R_1_last,S_3_first,S_3_mean,S_3_std,S_3_min,S_3_max,S_3_last,D_41_first,D_41_mean,D_41_std,D_41_min,D_41_max,D_41_last,B_3_first,B_3_mean,B_3_std,B_3_min,B_3_max,B_3_last,D_42_first,D_42_mean,D_42_std,D_42_min,D_42_max,D_42_last,D_43_first,D_43_mean,D_43_std,D_43_min,D_43_max,D_43_last,D_44_first,D_44_mean,D_44_std,D_44_min,D_44_max,D_44_last,B_4_first,B_4_mean,B_4_std,B_4_min,B_4_max,B_4_last,D_45_first,D_45_mean,D_45_std,D_45_min,D_45_max,D_45_last,B_5_first,B_5_mean,B_5_std,B_5_min,B_5_max,B_5_last,R_2_first,R_2_mean,R_2_std,R_2_min,R_2_max,R_2_last,D_46_first,D_46_mean,D_46_std,D_46_min,D_46_max,D_46_last,D_47_first,D_47_mean,D_47_std,D_47_min,D_47_max,D_47_last,D_48_first,D_48_mean,D_48_std,D_48_min,D_48_max,D_48_last,D_49_first,D_49_mean,D_49_std,D_49_min,D_49_max,D_49_last,B_6_first,B_6_mean,B_6_std,B_6_min,B_6_max,B_6_last,B_7_first,B_7_mean,B_7_std,B_7_min,B_7_max,B_7_last,B_8_first,B_8_mean,B_8_std,B_8_min,B_8_max,B_8_last,D_50_first,D_50_mean,D_50_std,D_50_min,D_50_max,D_50_last,D_51_first,D_51_mean,D_51_std,D_51_min,D_51_max,D_51_last,B_9_first,B_9_mean,B_9_std,B_9_min,B_9_max,B_9_last,R_3_first,R_3_mean,R_3_std,R_3_min,R_3_max,R_3_last,D_52_first,D_52_mean,D_52_std,D_52_min,D_52_max,D_52_last,P_3_first,P_3_mean,P_3_std,P_3_min,P_3_max,P_3_last,B_10_first,B_10_mean,B_10_std,B_10_min,B_10_max,B_10_last,D_53_first,D_53_mean,D_53_std,D_53_min,D_53_max,D_53_last,S_5_first,S_5_mean,S_5_std,S_5_min,S_5_max,S_5_last,B_11_first,B_11_mean,B_11_std,B_11_min,B_11_max,B_11_last,S_6_first,S_6_mean,S_6_std,S_6_min,S_6_max,S_6_last,D_54_first,D_54_mean,D_54_std,D_54_min,D_54_max,D_54_last,R_4_first,R_4_mean,R_4_std,R_4_min,R_4_max,R_4_last,S_7_first,S_7_mean,S_7_std,S_7_min,S_7_max,S_7_last,B_12_first,B_12_mean,B_12_std,B_12_min,B_12_max,B_12_last,S_8_first,S_8_mean,S_8_std,S_8_min,S_8_max,S_8_last,D_55_first,D_55_mean,D_55_std,D_55_min,D_55_max,D_55_last,D_56_first,D_56_mean,D_56_std,D_56_min,D_56_max,D_56_last,B_13_first,B_13_mean,B_13_std,B_13_min,B_13_max,B_13_last,R_5_first,R_5_mean,R_5_std,...,D_133_last_lag_sub,D_133_last_lag_div,R_28_last_lag_sub,R_28_last_lag_div,D_134_last_lag_sub,D_134_last_lag_div,D_135_last_lag_sub,D_135_last_lag_div,D_136_last_lag_sub,D_136_last_lag_div,D_137_last_lag_sub,D_137_last_lag_div,D_138_last_lag_sub,D_138_last_lag_div,D_139_last_lag_sub,D_139_last_lag_div,D_140_last_lag_sub,D_140_last_lag_div,D_141_last_lag_sub,D_141_last_lag_div,D_142_last_lag_sub,D_142_last_lag_div,D_143_last_lag_sub,D_143_last_lag_div,D_144_last_lag_sub,D_144_last_lag_div,D_145_last_lag_sub,D_145_last_lag_div,B_30_count,B_30_first,B_30_last,B_30_nunique,B_38_count,B_38_first,B_38_last,B_38_nunique,D_114_count,D_114_first,D_114_last,D_114_nunique,D_116_count,D_116_first,D_116_last,D_116_nunique,D_117_count,D_117_first,D_117_last,D_117_nunique,D_120_count,D_120_first,D_120_last,D_120_nunique,D_126_count,D_126_first,D_126_last,D_126_nunique,D_63_count,D_63_first,D_63_last,D_63_nunique,D_64_count,D_64_first,D_64_last,D_64_nunique,D_66_count,D_66_first,D_66_last,D_66_nunique,D_68_count,D_68_first,D_68_last,D_68_nunique,P_2_diff1,D_39_diff1,B_1_diff1,B_2_diff1,R_1_diff1,S_3_diff1,D_41_diff1,B_3_diff1,D_42_diff1,D_43_diff1,D_44_diff1,B_4_diff1,D_45_diff1,B_5_diff1,R_2_diff1,D_46_diff1,D_47_diff1,D_48_diff1,D_49_diff1,B_6_diff1,B_7_diff1,B_8_diff1,D_50_diff1,D_51_diff1,B_9_diff1,R_3_diff1,D_52_diff1,P_3_diff1,B_10_diff1,D_53_diff1,S_5_diff1,B_11_diff1,S_6_diff1,D_54_diff1,R_4_diff1,S_7_diff1,B_12_diff1,S_8_diff1,D_55_diff1,D_56_diff1,B_13_diff1,R_5_diff1,D_58_diff1,S_9_diff1,B_14_diff1,D_59_diff1,D_60_diff1,D_61_diff1,B_15_diff1,S_11_diff1,D_62_diff1,D_65_diff1,B_16_diff1,B_17_diff1,B_18_diff1,B_19_diff1,B_20_diff1,S_12_diff1,R_6_diff1,S_13_diff1,B_21_diff1,D_69_

In [62]:
def feature_filter(df, threshold=0.98):
    filtered_features = [col for col in df.columns if df[col].isnull().sum()/len(df) < threshold]
    return filtered_features

In [66]:
lst = feature_filter(train)

In [70]:
print(lst)

['customer_ID', 'P_2_first', 'P_2_mean', 'P_2_std', 'P_2_min', 'P_2_max', 'P_2_last', 'D_39_first', 'D_39_mean', 'D_39_std', 'D_39_min', 'D_39_max', 'D_39_last', 'B_1_first', 'B_1_mean', 'B_1_std', 'B_1_min', 'B_1_max', 'B_1_last', 'B_2_first', 'B_2_mean', 'B_2_std', 'B_2_min', 'B_2_max', 'B_2_last', 'R_1_first', 'R_1_mean', 'R_1_std', 'R_1_min', 'R_1_max', 'R_1_last', 'S_3_first', 'S_3_mean', 'S_3_std', 'S_3_min', 'S_3_max', 'S_3_last', 'D_41_first', 'D_41_mean', 'D_41_std', 'D_41_min', 'D_41_max', 'D_41_last', 'B_3_first', 'B_3_mean', 'B_3_std', 'B_3_min', 'B_3_max', 'B_3_last', 'D_42_first', 'D_42_mean', 'D_42_std', 'D_42_min', 'D_42_max', 'D_42_last', 'D_43_first', 'D_43_mean', 'D_43_std', 'D_43_min', 'D_43_max', 'D_43_last', 'D_44_first', 'D_44_mean', 'D_44_std', 'D_44_min', 'D_44_max', 'D_44_last', 'B_4_first', 'B_4_mean', 'B_4_std', 'B_4_min', 'B_4_max', 'B_4_last', 'D_45_first', 'D_45_mean', 'D_45_std', 'D_45_min', 'D_45_max', 'D_45_last', 'B_5_first', 'B_5_mean', 'B_5_std', 'B

In [72]:
len(lst)

1413

In [71]:
lst.pop(-1)

'target'

In [63]:
train = train[feature_filter(train)]
test = test[feature_filter(train)]

KeyError: "['target'] not in index"

In [42]:
train.head()

,customer_ID,P_2_first,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_first,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,B_1_first,B_1_mean,B_1_std,B_1_min,B_1_max,B_1_last,B_2_first,B_2_mean,B_2_std,B_2_min,B_2_max,B_2_last,R_1_first,R_1_mean,R_1_std,R_1_min,R_1_max,R_1_last,S_3_first,S_3_mean,S_3_std,S_3_min,S_3_max,S_3_last,D_41_first,D_41_mean,D_41_std,D_41_min,D_41_max,D_41_last,B_3_first,B_3_mean,B_3_std,B_3_min,B_3_max,B_3_last,D_42_first,D_42_mean,D_42_std,D_42_min,D_42_max,D_42_last,D_43_first,D_43_mean,D_43_std,D_43_min,D_43_max,D_43_last,D_44_first,D_44_mean,D_44_std,D_44_min,D_44_max,D_44_last,B_4_first,B_4_mean,B_4_std,B_4_min,B_4_max,B_4_last,D_45_first,D_45_mean,D_45_std,D_45_min,D_45_max,D_45_last,B_5_first,B_5_mean,B_5_std,B_5_min,B_5_max,B_5_last,R_2_first,R_2_mean,R_2_std,R_2_min,R_2_max,R_2_last,D_46_first,D_46_mean,D_46_std,D_46_min,D_46_max,D_46_last,D_47_first,D_47_mean,D_47_std,D_47_min,D_47_max,D_47_last,D_48_first,D_48_mean,D_48_std,D_48_min,D_48_max,D_48_last,D_49_first,D_49_mean,D_49_std,D_49_min,D_49_max,D_49_last,B_6_first,B_6_mean,B_6_std,B_6_min,B_6_max,B_6_last,B_7_first,B_7_mean,B_7_std,B_7_min,B_7_max,B_7_last,B_8_first,B_8_mean,B_8_std,B_8_min,B_8_max,B_8_last,D_50_first,D_50_mean,D_50_std,D_50_min,D_50_max,D_50_last,D_51_first,D_51_mean,D_51_std,D_51_min,D_51_max,D_51_last,B_9_first,B_9_mean,B_9_std,B_9_min,B_9_max,B_9_last,R_3_first,R_3_mean,R_3_std,R_3_min,R_3_max,R_3_last,D_52_first,D_52_mean,D_52_std,D_52_min,D_52_max,D_52_last,P_3_first,P_3_mean,P_3_std,P_3_min,P_3_max,P_3_last,B_10_first,B_10_mean,B_10_std,B_10_min,B_10_max,B_10_last,D_53_first,D_53_mean,D_53_std,D_53_min,D_53_max,D_53_last,S_5_first,S_5_mean,S_5_std,S_5_min,S_5_max,S_5_last,B_11_first,B_11_mean,B_11_std,B_11_min,B_11_max,B_11_last,S_6_first,S_6_mean,S_6_std,S_6_min,S_6_max,S_6_last,D_54_first,D_54_mean,D_54_std,D_54_min,D_54_max,D_54_last,R_4_first,R_4_mean,R_4_std,R_4_min,R_4_max,R_4_last,S_7_first,S_7_mean,S_7_std,S_7_min,S_7_max,S_7_last,B_12_first,B_12_mean,B_12_std,B_12_min,B_12_max,B_12_last,S_8_first,S_8_mean,S_8_std,S_8_min,S_8_max,S_8_last,D_55_first,D_55_mean,D_55_std,D_55_min,D_55_max,D_55_last,D_56_first,D_56_mean,D_56_std,D_56_min,D_56_max,D_56_last,B_13_first,B_13_mean,B_13_std,B_13_min,B_13_max,B_13_last,R_5_first,R_5_mean,R_5_std,...,D_130_last_lag_sub,D_130_last_lag_div,D_131_last_lag_sub,D_131_last_lag_div,D_132_last_lag_sub,D_132_last_lag_div,D_133_last_lag_sub,D_133_last_lag_div,R_28_last_lag_sub,D_134_last_lag_sub,D_134_last_lag_div,D_135_last_lag_sub,D_135_last_lag_div,D_136_last_lag_sub,D_136_last_lag_div,D_137_last_lag_sub,D_137_last_lag_div,D_138_last_lag_sub,D_138_last_lag_div,D_139_last_lag_sub,D_139_last_lag_div,D_140_last_lag_sub,D_140_last_lag_div,D_141_last_lag_sub,D_141_last_lag_div,D_142_last_lag_sub,D_142_last_lag_div,D_143_last_lag_sub,D_143_last_lag_div,D_144_last_lag_sub,D_144_last_lag_div,D_145_last_lag_sub,D_145_last_lag_div,B_30_count,B_30_first,B_30_last,B_30_nunique,B_38_count,B_38_first,B_38_last,B_38_nunique,D_114_count,D_114_first,D_114_last,D_114_nunique,D_116_count,D_116_first,D_116_last,D_116_nunique,D_117_count,D_117_first,D_117_last,D_117_nunique,D_120_count,D_120_first,D_120_last,D_120_nunique,D_126_count,D_126_first,D_126_last,D_126_nunique,D_63_count,D_63_first,D_63_last,D_63_nunique,D_64_count,D_64_first,D_64_last,D_64_nunique,D_66_count,D_66_first,D_66_last,D_66_nunique,D_68_count,D_68_first,D_68_last,D_68_nunique,P_2_diff1,D_39_diff1,B_1_diff1,B_2_diff1,R_1_diff1,S_3_diff1,D_41_diff1,B_3_diff1,D_42_diff1,D_43_diff1,D_44_diff1,B_4_diff1,D_45_diff1,B_5_diff1,R_2_diff1,D_46_diff1,D_47_diff1,D_48_diff1,D_49_diff1,B_6_diff1,B_7_diff1,B_8_diff1,D_50_diff1,D_51_diff1,B_9_diff1,R_3_diff1,D_52_diff1,P_3_diff1,B_10_diff1,D_53_diff1,S_5_diff1,B_11_diff1,S_6_diff1,D_54_diff1,R_4_diff1,S_7_diff1,B_12_diff1,S_8_diff1,D_55_diff1,D_56_diff1,B_13_diff1,R_5_diff1,D_58_diff1,S_9_diff1,B_14_diff1,D_59_diff1,D_60_diff1,D_61_diff1,B_15_diff1,S_11_diff1,D_62_diff1,D_65_diff1,B_16_di

In [86]:
def xgb_amex(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred)

def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True


# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

In [87]:
# Label encode categorical features
cat_features = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68"
]
cat_features = [f"{cf}_last" for cf in cat_features]
for cat_col in cat_features:
    encoder = LabelEncoder()
    train[cat_col] = encoder.fit_transform(train[cat_col])
    test[cat_col] = encoder.transform(test[cat_col])
# Round last float features to 2 decimal place
num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
num_cols = [col for col in num_cols if 'last' in col]
for col in num_cols:
    try:
        train[col + '_round2'] = train[col].round(2)
        test[col + '_round2'] = test[col].round(2)
    except:
        pass
# Get the difference between last and mean
num_cols = [col for col in train.columns if 'last' in col]
num_cols = [col[:-5] for col in num_cols if 'round' not in col]
for col in num_cols:
    try:
        train[f'{col}_last_mean_diff'] = train[f'{col}_last'] - train[f'{col}_mean']
        test[f'{col}_last_mean_diff'] = test[f'{col}_last'] - test[f'{col}_mean']
    except:
        pass
'''# Transform float64 and float32 to float16
num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
for col in tqdm(num_cols):
    train[col] = train[col].astype(np.float16)
    test[col] = test[col].astype(np.float16)'''

"# Transform float64 and float32 to float16\nnum_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)\nfor col in tqdm(num_cols):\n    train[col] = train[col].astype(np.float16)\n    test[col] = test[col].astype(np.float16)"

In [89]:
test.head()

,customer_ID,P_2_first,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_first,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,B_1_first,B_1_mean,B_1_std,B_1_min,B_1_max,B_1_last,B_2_first,B_2_mean,B_2_std,B_2_min,B_2_max,B_2_last,R_1_first,R_1_mean,R_1_std,R_1_min,R_1_max,R_1_last,S_3_first,S_3_mean,S_3_std,S_3_min,S_3_max,S_3_last,D_41_first,D_41_mean,D_41_std,D_41_min,D_41_max,D_41_last,B_3_first,B_3_mean,B_3_std,B_3_min,B_3_max,B_3_last,D_42_first,D_42_mean,D_42_std,D_42_min,D_42_max,D_42_last,D_43_first,D_43_mean,D_43_std,D_43_min,D_43_max,D_43_last,D_44_first,D_44_mean,D_44_std,D_44_min,D_44_max,D_44_last,B_4_first,B_4_mean,B_4_std,B_4_min,B_4_max,B_4_last,D_45_first,D_45_mean,D_45_std,D_45_min,D_45_max,D_45_last,B_5_first,B_5_mean,B_5_std,B_5_min,B_5_max,B_5_last,R_2_first,R_2_mean,R_2_std,R_2_min,R_2_max,R_2_last,D_46_first,D_46_mean,D_46_std,D_46_min,D_46_max,D_46_last,D_47_first,D_47_mean,D_47_std,D_47_min,D_47_max,D_47_last,D_48_first,D_48_mean,D_48_std,D_48_min,D_48_max,D_48_last,D_49_first,D_49_mean,D_49_std,D_49_min,D_49_max,D_49_last,B_6_first,B_6_mean,B_6_std,B_6_min,B_6_max,B_6_last,B_7_first,B_7_mean,B_7_std,B_7_min,B_7_max,B_7_last,B_8_first,B_8_mean,B_8_std,B_8_min,B_8_max,B_8_last,D_50_first,D_50_mean,D_50_std,D_50_min,D_50_max,D_50_last,D_51_first,D_51_mean,D_51_std,D_51_min,D_51_max,D_51_last,B_9_first,B_9_mean,B_9_std,B_9_min,B_9_max,B_9_last,R_3_first,R_3_mean,R_3_std,R_3_min,R_3_max,R_3_last,D_52_first,D_52_mean,D_52_std,D_52_min,D_52_max,D_52_last,P_3_first,P_3_mean,P_3_std,P_3_min,P_3_max,P_3_last,B_10_first,B_10_mean,B_10_std,B_10_min,B_10_max,B_10_last,D_53_first,D_53_mean,D_53_std,D_53_min,D_53_max,D_53_last,S_5_first,S_5_mean,S_5_std,S_5_min,S_5_max,S_5_last,B_11_first,B_11_mean,B_11_std,B_11_min,B_11_max,B_11_last,S_6_first,S_6_mean,S_6_std,S_6_min,S_6_max,S_6_last,D_54_first,D_54_mean,D_54_std,D_54_min,D_54_max,D_54_last,R_4_first,R_4_mean,R_4_std,R_4_min,R_4_max,R_4_last,S_7_first,S_7_mean,S_7_std,S_7_min,S_7_max,S_7_last,B_12_first,B_12_mean,B_12_std,B_12_min,B_12_max,B_12_last,S_8_first,S_8_mean,S_8_std,S_8_min,S_8_max,S_8_last,D_55_first,D_55_mean,D_55_std,D_55_min,D_55_max,D_55_last,D_56_first,D_56_mean,D_56_std,D_56_min,D_56_max,D_56_last,B_13_first,B_13_mean,B_13_std,B_13_min,B_13_max,B_13_last,R_5_first,R_5_mean,R_5_std,...,D_102_last_lag_div_round2,D_103_last_lag_div_round2,D_104_last_lag_sub_round2,D_104_last_lag_div_round2,D_105_last_lag_sub_round2,D_105_last_lag_div_round2,D_106_last_lag_div_round2,D_107_last_lag_div_round2,B_36_last_lag_sub_round2,B_36_last_lag_div_round2,B_37_last_lag_sub_round2,B_37_last_lag_div_round2,R_26_last_lag_div_round2,R_27_last_lag_sub_round2,R_27_last_lag_div_round2,D_108_last_lag_div_round2,D_109_last_lag_div_round2,D_110_last_lag_sub_round2,D_110_last_lag_div_round2,D_111_last_lag_div_round2,B_39_last_lag_sub_round2,B_39_last_lag_div_round2,D_112_last_lag_sub_round2,D_112_last_lag_div_round2,B_40_last_lag_sub_round2,B_40_last_lag_div_round2,S_27_last_lag_sub_round2,S_27_last_lag_div_round2,D_113_last_lag_div_round2,D_115_last_lag_sub_round2,D_115_last_lag_div_round2,D_118_last_lag_sub_round2,D_118_last_lag_div_round2,D_119_last_lag_sub_round2,D_119_last_lag_div_round2,D_121_last_lag_sub_round2,D_121_last_lag_div_round2,D_122_last_lag_div_round2,D_123_last_lag_div_round2,D_124_last_lag_div_round2,D_125_last_lag_div_round2,D_127_last_lag_div_round2,D_128_last_lag_sub_round2,D_128_last_lag_div_round2,D_129_last_lag_div_round2,B_41_last_lag_div_round2,B_42_last_lag_sub_round2,B_42_last_lag_div_round2,D_130_last_lag_sub_round2,D_130_last_lag_div_round2,D_131_last_lag_sub_round2,D_131_last_lag_div_round2,D_132_last_lag_sub_round2,D_132_last_lag_div_round2,D_133_last_lag_sub_round2,D_133_last_lag_div_round2,R_28_last_lag_div_round2,D_134_last_lag_sub_round2,D_134_last_lag_div_round2,D_135_last_lag_div_round2,D_136_last_lag_div_round2,D_137_last_lag_div_round2,D_138_last_lag_div_round2,D_139_last_lag_div_round2,D_140_last_lag_div_round2,D_141_last_lag_sub_round2

In [90]:
def xgb_train(x_train, y_train, x_val, y_val):
    print("# of features:", x_train.shape[1])
    assert x_train.shape[1] == x_val.shape[1]
    dtrain = xgb.DMatrix(data=x_train, label=y_train)
    dvalid = xgb.DMatrix(data=x_val, label=y_val)
    params = {
            #'booster': 'dart',
            #'rate_drop': 0.1,
            #'skip_drop': 0.5,
            'objective': 'binary:logistic',
            'tree_method': 'gpu_hist',
            'max_depth': 7,
            'subsample':0.88, #0.88
            'colsample_bytree': 0.5,
            'gamma':1.5,
            'min_child_weight':8,
            'lambda':70,
            'eta':0.03, #0.03
    }
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    model = xgb.train(params, dtrain=dtrain,
                num_boost_round=10500,evals=watchlist,
                early_stopping_rounds=2000, feval=xgb_amex, maximize=True,
                verbose_eval=100)   #num_boost_round=2600, early_stopping_rounds = 500 , feval=xgb_amex
    print('best tree_limit:', model.best_ntree_limit)
    print('best score:', model.best_score)
    return model.predict(dvalid, iteration_range=(0,model.best_ntree_limit)), model
seed_everything(CFG.seed)

In [91]:
# Get feature list
features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]
test_data = test[features]
dtest = xgb.DMatrix(data=test_data)

In [92]:
# Create a numpy array to store test predictions
test_predictions = np.zeros(len(test))
# Create a numpy array to store out of folds predictions
oof_predictions = np.zeros(len(train))

kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = 42)
for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
    print(' ')
    print('-'*50)
    print(f'Training fold {fold} with {len(features)} features...')
    x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
    y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]

    # x = x_train, y = y_train, xt = x_val, yt = y_val

    val_pred, model = xgb_train(x_train, y_train, x_val, y_val)
    model.save_model(OUTPUT_DIR+f'{expt_name}_fold{fold}.xgb')
    # Add to out of folds array
    oof_predictions[val_ind] = val_pred

    # Predict the test set
    test_pred = model.predict(dtest)
    test_predictions += test_pred / CFG.n_folds

    amex_score = amex_metric(y_val, val_pred)

    print(f'Fold {fold} CV score is {amex_score}')

# Compute out of folds metric
score = amex_metric(train[CFG.target], oof_predictions)
print(f'Our out of folds CV score is {score}')

# Create a dataframe to store out of folds predictions
oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
oof_df.to_csv(OUTPUT_DIR+f'oof_{expt_name}_seed{CFG.seed}.csv', index = False)
# Create a dataframe to store test prediction
test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
test_df.to_csv(OUTPUT_DIR+f'test_{expt_name}_seed{CFG.seed}.csv', index = False)


 
--------------------------------------------------
Training fold 0 with 2177 features...
# of features: 2177
[0]	train-logloss:0.67374	train-amex_metric:0.70957	eval-logloss:0.67380	eval-amex_metric:0.70497
[100]	train-logloss:0.24152	train-amex_metric:0.78181	eval-logloss:0.24621	eval-amex_metric:0.77007
[200]	train-logloss:0.21586	train-amex_metric:0.80031	eval-logloss:0.22491	eval-amex_metric:0.78461
[300]	train-logloss:0.20667	train-amex_metric:0.81356	eval-logloss:0.22002	eval-amex_metric:0.78978
[400]	train-logloss:0.20007	train-amex_metric:0.82426	eval-logloss:0.21777	eval-amex_metric:0.79278
[500]	train-logloss:0.19469	train-amex_metric:0.83271	eval-logloss:0.21650	eval-amex_metric:0.79512
[600]	train-logloss:0.19024	train-amex_metric:0.84031	eval-logloss:0.21567	eval-amex_metric:0.79642
[700]	train-logloss:0.18612	train-amex_metric:0.84803	eval-logloss:0.21513	eval-amex_metric:0.79710
[800]	train-logloss:0.18222	train-amex_metric:0.85467	eval-logloss:0.21473	eval-amex_metric